<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#housekeeping" data-toc-modified-id="housekeeping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>housekeeping</a></span><ul class="toc-item"><li><span><a href="#languages-&amp;-packages" data-toc-modified-id="languages-&amp;-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>languages &amp; packages</a></span></li><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>env set-up</a></span></li><li><span><a href="#model-function" data-toc-modified-id="model-function-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>model function</a></span></li></ul></li><li><span><a href="#step-by-step-numerical-solve-via-SymPy" data-toc-modified-id="step-by-step-numerical-solve-via-SymPy-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>step-by-step numerical solve via SymPy</a></span><ul class="toc-item"><li><span><a href="#environmental-preparation" data-toc-modified-id="environmental-preparation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>environmental preparation</a></span></li><li><span><a href="#solve-each-parameter-in-each-equation" data-toc-modified-id="solve-each-parameter-in-each-equation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>solve each parameter in each equation</a></span><ul class="toc-item"><li><span><a href="#$g_p$:-in-$dp$-&amp;-$dm$" data-toc-modified-id="$g_p$:-in-$dp$-&amp;-$dm$-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>$g_p$: in $dp$ &amp; $dm$</a></span></li><li><span><a href="#$e_p$:-in-$dp$-&amp;-$dm$" data-toc-modified-id="$e_p$:-in-$dp$-&amp;-$dm$-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>$e_p$: in $dp$ &amp; $dm$</a></span></li><li><span><a href="#$R_p$:-in-$dp$" data-toc-modified-id="$R_p$:-in-$dp$-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>$R_p$: in $dp$</a></span></li><li><span><a href="#$m_p$:-in-$dp$-&amp;-$dm$" data-toc-modified-id="$m_p$:-in-$dp$-&amp;-$dm$-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>$m_p$: in $dp$ &amp; $dm$</a></span></li><li><span><a href="#$g_c$:-in-$dc$-&amp;-$dm$" data-toc-modified-id="$g_c$:-in-$dc$-&amp;-$dm$-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>$g_c$: in $dc$ &amp; $dm$</a></span></li><li><span><a href="#$e_c$:-in-$dc$-&amp;-$dm$" data-toc-modified-id="$e_c$:-in-$dc$-&amp;-$dm$-2.2.6"><span class="toc-item-num">2.2.6&nbsp;&nbsp;</span>$e_c$: in $dc$ &amp; $dm$</a></span></li><li><span><a href="#$R_c$:-in-$dc$" data-toc-modified-id="$R_c$:-in-$dc$-2.2.7"><span class="toc-item-num">2.2.7&nbsp;&nbsp;</span>$R_c$: in $dc$</a></span></li><li><span><a href="#$m_c$:-in-$dc$-&amp;-$dm$" data-toc-modified-id="$m_c$:-in-$dc$-&amp;-$dm$-2.2.8"><span class="toc-item-num">2.2.8&nbsp;&nbsp;</span>$m_c$: in $dc$ &amp; $dm$</a></span></li></ul></li></ul></li><li><span><a href="#solve-all-parameters-in-one-go" data-toc-modified-id="solve-all-parameters-in-one-go-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>solve all parameters in one go</a></span></li></ul></div>

# housekeeping
This notebook is for analytically solving parameters in the set of ODEs for eco-bioelectric cell (EBC).

## languages & packages
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

## env set-up

In [1]:
using PyCall, DataFrames, SymPy
sympy.init_printing() ## use other py3 function not implemented in native Julia-lang
println()

In [2]:
# function ebc0(popDen, t, gp,ep,Rp,mp, gc,ec,Rc,mc)
    
#     ## population densities
#     P = popDen[:1]
#     M = popDen[:2]
#     C = popDen[:3]
    
#     ## numerical adaptation to net growth
#     #if gp <1; gp = 1+gp;end
#     #if gc <1; gc = 1+gc;end
    
#     ## fluctuation model
#     dp = P*(gp*ep - Rp - mp)
#     dm = P*(gp*(1-ep) + mp) + C*(gc*(1-ec)*M + mc)
#     dc = C*(gc*ec*M - Rc - mc)
    
#     ## non-recoverable elimination
#     if P <=0; dp=0;end
#     if C <=0; dc=0;end
    
#     return(sc.array([dp,dm,dc]))
# end

ebc0 (generic function with 1 method)

# step-by-step numerical solve via SymPy
This step-wise workflow is following CMEE [notes](https://nbviewer.jupyter.org/github/mhasoba/TheMulQuaBio/blob/master/notebooks/Appendix-Maths.ipynb) (last section) from Samraat on Jupyter math models
## environmental preparation
define variables

In [3]:
@vars P; @vars M; @vars C
@vars gp; @vars ep; @vars Rp; @vars mp
@vars gc; @vars ec; @vars Rc; @vars mc

(mc,)

define SymPy ODE system

In [4]:
dp = P*(gp*ep - Rp - mp)
dm = P*(gp*(1-ep) + mp) + C*(gc*(1-ec)*M + mc)
dc = C*(gc*ec*M - Rc - mc)
dp, dm, dc

(P*(-Rp + ep*gp - mp), C*(M*gc*(1 - ec) + mc) + P*(gp*(1 - ep) + mp), C*(M*ec*gc - Rc - mc))

In [5]:
dp

P⋅(-Rp + ep⋅gp - mp)

In [6]:
dm

C⋅(M⋅gc⋅(1 - ec) + mc) + P⋅(gp⋅(1 - ep) + mp)

In [7]:
dc

C⋅(M⋅ec⋅gc - Rc - mc)

define eqm state

In [8]:
p_eqm = Eq(dp, 0)
m_eqm = Eq(dm, 0)
c_eqm = Eq(dc, 0)
p_eqm, m_eqm, c_eqm

(Eq(P*(-Rp + ep*gp - mp), 0), Eq(C*(M*gc*(1 - ec) + mc) + P*(gp*(1 - ep) + mp), 0), Eq(C*(M*ec*gc - Rc - mc), 0))

In [9]:
p_eqm

P⋅(-Rp + ep⋅gp - mp) = 0

In [10]:
m_eqm

C⋅(M⋅gc⋅(1 - ec) + mc) + P⋅(gp⋅(1 - ep) + mp) = 0

In [11]:
c_eqm

C⋅(M⋅ec⋅gc - Rc - mc) = 0

# solve all parameters in one go
summary: there's no co-existence equilibrium position

In [38]:
solve([p_eqm, m_eqm, c_eqm], [P,M,C])

3-element Array{NTuple{11,Sym},1}:
 (0, mc/(gc*(ec - 1)), C, gp, ep, Rp, mp, gc, ec, mc/(ec - 1), mc)                                        
 (P, M, 0, gp, (gp + mp)/gp, gp, mp, gc, ec, Rc, mc)                                                      
 (P, (Rc + mc)/(ec*gc), -P*ec*(ep*gp - gp - mp)/(Rc*ec - Rc - mc), gp, ep, ep*gp - mp, mp, gc, ec, Rc, mc)